In [123]:

import scipy.io
import torch
import importlib
import matplotlib.pyplot as plt
import numpy as np

import src.HyperParameters as hp 
import src.data as dt 
import src.model3 as Model 
import src.train as trainer

importlib.reload(hp)
importlib.reload(dt)
importlib.reload(Model)
importlib.reload(trainer)



<module 'src.train' from '/Users/ramtarun/Desktop/Cambridge/Friction-Factor-Estimation-PINN/src/train.py'>

In [124]:
def list_available_devices():
    devices = []
    if torch.cuda.is_available():
        devices.append('cuda')
    if torch.cuda.is_available() and torch.cuda.device_count() > 1:
        devices.append('multi-gpu')
    if torch.cuda.is_available() and torch.cuda.nccl.version() > 0:
        devices.append('nccl')
    if torch.cuda.is_available() and torch.cuda.is_initialized():
        devices.append('initialized')
    if torch.cuda.is_available() and torch.cuda.memory_allocated() > 0:
        devices.append('memory-allocated')
    if torch.cuda.is_available() and torch.cuda.memory_reserved() > 0:
        devices.append('memory-reserved')
    devices.append('cpu')
    devices.append('mps')

    return devices

def get_device_choice():
    devices = list_available_devices()
    print("Available devices:")
    for i, device in enumerate(devices):
        print(f"{i + 1}. {device}")
    while True:
        choice = input("Choose the device for model training (enter the corresponding number): ")
        if choice.isdigit() and 1 <= int(choice) <= len(devices):
            return devices[int(choice) - 1]
        print("Invalid choice. Please enter a valid device number.")

# Usage
device = get_device_choice()
print(f"Selected device: {device}")

Available devices:
1. cpu
2. tpu
3. mps
Selected device: mps


In [125]:
data = scipy.io.loadmat('/Users/ramtarun/Desktop/Cambridge/Indirect-Noise-in-Nozzles/Data/Data_PINN_subsonic_geom_linvelsup_f0-0.1.mat')

In [126]:
PINN_model = Model.PINN(hp.input_size, hp.output_size, hp.hidden_size, hp.num_layers, hp.lda)
PINN_model.to(device, non_blocking=False)

PINN(
  (loss_function): MSELoss()
  (rnn): PhyGRU(
    (gru): GRU(2, 4, num_layers=2, batch_first=True, dropout=0.5)
    (output_layer): Linear(in_features=4, out_features=4, bias=True)
    (activation): Tanh()
  )
)

In [127]:
params = list(PINN_model.parameters())
# optimizer = torch.optim.Adam([{'params' : params[1::]},{'params' : params[-1], 'lr': hp.ff_learning_rate}], lr = hp.learning_rate, amsgrad = True)   
optimizer = torch.optim.Adam(params=params, lr = hp.learning_rate, amsgrad = True)   
# optimizer = torch.optim.LBFGS(params, hp.ff_learning_rate, 
#                               max_iter = hp.epochs, 
#                               max_eval = None, 
#                               tolerance_grad = 1e-11, 
#                               tolerance_change = 1e-11, 
#                               history_size = 100, 
#                               line_search_fn = 'strong_wolfe')


In [128]:
device

'mps'

In [129]:
inputs, targets, meanflow =  dt.DataPreprocessing(data, ff=0.01, device = device)
inputs.to(device)
targets.to(device)
meanflow.to(device)

N = inputs.shape[1]
train_loader, val_loader = dt.DataTransformer(inputs, targets, meanflow, TrainingSet=True)

meanflow.device

device(type='mps', index=0)

In [130]:
fval = [0.1  , 0.06 , 0.04, 0.01]

# for f in fval:
# ### Model 3
# batch_size = 32
# sequence_length = len(inputs) // batch_size
# # Reshape the input tensors within the DataLoader
# train_loader = []
# for batch in Train_loader:
#     input_data = batch.view(batch_size, sequence_length, 1)
#     train_loader.append(input_data)

train_loss, val_loss, f_train, f_test, f_dist = trainer.train(train_loader, val_loader, hp.epochs, optimizer, PINN_model, N)

with torch.no_grad():
    plt.figure() 
    plt.plot(train_loss.keys(), train_loss.values(), 'r-', label='Training Loss')
    plt.plot(val_loss.keys(), val_loss.values(), 'g-', label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid()
    plt.show()
    plt.figure()
    plt.plot(f_train.keys(), f_train.values(), 'r-', label='Training Friction Factor')
    plt.plot(f_test.keys(), f_test.values(), 'g-', label='Validation Friction Factor')
    plt.xlabel('Epochs')
    plt.ylabel('Friction Factor')
    plt.legend()
    plt.grid()
    plt.show()
    plt.figure()


Epoch 5/500 - Train Loss: 16.207254 Val Loss: 0.534402 f_train: -0.1256 f_test: -0.1226
Epoch 10/500 - Train Loss: 14.792564 Val Loss: 0.488329 f_train: -0.1152 f_test: -0.1104
Epoch 15/500 - Train Loss: 13.387316 Val Loss: 0.441396 f_train: -0.1185 f_test: -0.1163
Epoch 20/500 - Train Loss: 12.044244 Val Loss: 0.398381 f_train: -0.1189 f_test: -0.1098
Epoch 25/500 - Train Loss: 10.789747 Val Loss: 0.357223 f_train: -0.1132 f_test: -0.1068
Epoch 30/500 - Train Loss: 9.647078 Val Loss: 0.316306 f_train: -0.0959 f_test: -0.0991
Epoch 35/500 - Train Loss: 8.640535 Val Loss: 0.281520 f_train: -0.0861 f_test: -0.0838
Epoch 40/500 - Train Loss: 7.846201 Val Loss: 0.257969 f_train: -0.0689 f_test: -0.0573
Epoch 45/500 - Train Loss: 7.408281 Val Loss: 0.244384 f_train: -0.0250 f_test: -0.0298
Epoch 50/500 - Train Loss: 7.626735 Val Loss: 0.246262 f_train: 0.0114 f_test: 0.0039
Epoch 55/500 - Train Loss: 96.269034 Val Loss: 0.251805 f_train: 0.0249 f_test: 0.0327


RuntimeError: The total norm of order 2.0 for gradients from `parameters` is non-finite, so it cannot be clipped. To disable this error and scale the gradients by the non-finite norm anyway, set `error_if_nonfinite=False`

In [ ]:
def transform_sequence(tensor, seq_length):
    # tensor shape: (N, 2)
    # seq_length: desired sequence length
  

    # Add a new dimension of size 1 to the tensor
    tensor = torch.unsqueeze(tensor, dim=1)

    # Repeat the tensor along the added dimension to match the desired sequence length
    tensor = tensor.repeat(1, seq_length, 1)

    # tensor shape: (N, L, 2)
    return tensor